# Water Jug Problem

This notebook shows how to set up and solve a Water Jug Problem as an Integer Linear Program (ILP)

## What is the Water Jug Problem?

Suppose you have the following:
* An empty 5 gallon jug (Jug A)
* An empty 3 gallon jug (Jug B)
* An "infinite" water source like a spring

Pour water into the jugs such that you end up with:
* The 5 gallon jug containing 4 gallons
* The 3 gallon jug containing 3 gallons

You're allowed to make the following moves:
* Dump out all of the water Jug A
* Dump out all of the water Jug B
* Completely fill up Jug A
* Completely fill up Jug B
* Pout water from one jug into the other

Which decisions (pours/dumps/fills) should you make such that the total number of decisions is minimized?

# Imports

In [1]:
from docplex.mp.model import Model

# Sets

In [2]:
# set of time periods
T = list(range(11))

# set of jugs
J = ['A','B']

# capacity of jugs
C = {
    'A':5,
    'B':3
}

# set of actions
decisions = {
    0: 'Dump A',
    1: 'Dump B',
    2: 'Fill A',
    3: 'Fill B',
    4: 'Pour A to B',
    5: 'Pour B to A'
}

D = list(range(6))


# Decision Variables

In [3]:
# create a model object
water_jug = Model(name='Water-Jug-Problem')

In [4]:
# variables that equal 1 if we make a move and 0 otherwise
z = water_jug.binary_var_dict(keys = [(d, t) for d in D for t in T[1:]], name="z")

In [5]:
def get_ub(var_name):
    if var_name[0] == 'A':
        ub = 5
    else:
        ub = 3
    return ub

In [6]:
# volume of water in jugs
v = water_jug.integer_var_dict(keys = [(j, t) for j in J for t in T], name="v",lb=0, ub=get_ub)

# Objective Function

In [7]:
# add the objective function of minimizing total distance
num_moves = water_jug.sum(z[(d,t)] for d in D for t in T[1:])
water_jug.minimize(num_moves)

# Constraints

In [8]:
# initial volumes are zero for both jugs
water_jug.add_constraints(v[j,0] == 0 for j in J)

[docplex.mp.LinearConstraint[](v_A_0,EQ,0),
 docplex.mp.LinearConstraint[](v_B_0,EQ,0)]

In [9]:
# at most a single decision can be made at each time period
water_jug.add_constraints(water_jug.sum(z[(d,t)] for d in D) <= 1 for t in T[1:])

[docplex.mp.LinearConstraint[](z_0_1+z_1_1+z_2_1+z_3_1+z_4_1+z_5_1,LE,1),
 docplex.mp.LinearConstraint[](z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2,LE,1),
 docplex.mp.LinearConstraint[](z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3,LE,1),
 docplex.mp.LinearConstraint[](z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4,LE,1),
 docplex.mp.LinearConstraint[](z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5,LE,1),
 docplex.mp.LinearConstraint[](z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6,LE,1),
 docplex.mp.LinearConstraint[](z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7,LE,1),
 docplex.mp.LinearConstraint[](z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8,LE,1),
 docplex.mp.LinearConstraint[](z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9,LE,1),
 docplex.mp.LinearConstraint[](z_0_10+z_1_10+z_2_10+z_3_10+z_4_10+z_5_10,LE,1)]

In [10]:
# add dumping/filling constraints
for t in T[1:]:

    # dump A
    water_jug.add_indicator(z[(0,t)], v[('A',t)]==0)

    # dump B
    water_jug.add_indicator(z[(1,t)], v[('B',t)]==0)

    # Fill A
    water_jug.add_indicator(z[(2,t)], v[('A',t)]==5)

    # Fill B
    water_jug.add_indicator(z[(3,t)], v[('B',t)]==3)

In [11]:
# a stays same if no movement
for t in T[1:]:
    water_jug.add_if_then(z[(0,t)] + z[(2,t)] + z[(4,t)] + z[(5,t)] == 0, v[('A',t)] == v[('A',t-1)])


In [12]:
# b stays same if no movement
for t in T[1:]:
    water_jug.add_if_then(z[(1,t)] + z[(3,t)] + z[(4,t)] + z[(5,t)] == 0, v[('B',t)] == v[('B',t-1)])

In [13]:
# if you pour A into B
for t in T[1:]:
    
    # for A
    water_jug.add_indicator(z[(4,t)], v[('A',t)]==water_jug.max(0, v[('A',t-1)]-3+v[('B',t-1)]  ))

    # for B
    water_jug.add_indicator(z[(4,t)], v[('B',t)]==water_jug.min(3, v[('A',t-1)]+ v[('B',t-1)]  ))

In [14]:
# if you pour B into A
for t in T[1:]:
    
    # for A
    water_jug.add_indicator(z[(5,t)], v[('A',t)]==water_jug.min(5, v[('A',t-1)]+ v[('B',t-1)]  ))

    # for B
    water_jug.add_indicator(z[(5,t)], v[('B',t)]==water_jug.max(0, v[('A',t-1)]+ v[('B',t-1)] -5 ))

In [15]:
# if you don't make a move on turn t, then no move on turn t-1
water_jug.add_constraints(water_jug.sum(z[(d,t+1)] for d in D) <= water_jug.sum(z[(d,t)] for d in D) for t in T[1:-1])

[docplex.mp.LinearConstraint[](z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2,LE,z_0_1+z_1_1+z_2_1+z_3_1+z_4_1+z_5_1),
 docplex.mp.LinearConstraint[](z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3,LE,z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2),
 docplex.mp.LinearConstraint[](z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4,LE,z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3),
 docplex.mp.LinearConstraint[](z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5,LE,z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4),
 docplex.mp.LinearConstraint[](z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6,LE,z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5),
 docplex.mp.LinearConstraint[](z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7,LE,z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6),
 docplex.mp.LinearConstraint[](z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8,LE,z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7),
 docplex.mp.LinearConstraint[](z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9,LE,z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8),
 docplex.mp.LinearConstraint[](z_0_10+z_1_10+z_2_10+z_3_10+z_4_10+z_5_10,LE,z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9)]

In [16]:
# end results are 4 and 3
water_jug.add_constraint(v['A',10] == 4)
water_jug.add_constraint(v['B',10] == 3)

docplex.mp.LinearConstraint[](v_B_10,EQ,3)

# Solve the Problem

In [17]:
water_jug.solve()

docplex.mp.solution.SolveSolution(obj=6,values={z_1_3:1,z_2_1:1,z_2_5:1,..

In [21]:
# set of actions
decisions = {
    0: 'Dump A',
    1: 'Dump B',
    2: 'Fill A',
    3: 'Fill B',
    4: 'Pour A to B',
    5: 'Pour B to A'
}

z_values = water_jug.solution.get_value_df(z)
z_values.loc[z_values.value == 1].sort_values(by=['key_2'])

,key_1,key_2,value
20,2,1,1.0
41,4,2,1.0
12,1,3,1.0
43,4,4,1.0
24,2,5,1.0
45,4,6,1.0


In [20]:
water_jug.solution.get_value_df(v)

,key_1,key_2,value
0,A,0,0.0
1,A,1,5.0
2,A,2,2.0
3,A,3,2.0
4,A,4,0.0
5,A,5,5.0
6,A,6,4.0
7,A,7,4.0
8,A,8,4.0
9,A,9,4.0


### 1. Solve an Assignment Problem

The following model is a linear program of the Assignment Problem

* $c_{ij}$: The cost of traveling from node $i$ to node $j$
* $x_{ij}$: A decision variable that equals $1$ if we travel from node $i$ directly to $j$ and $0$ otherwise

$min \quad \sum_{(i,j) \in I X J} c_{ij}x_{ij}$

$s.t.$

$\sum_{j \in J}x_{ij}=1, \quad \forall i \in I$

$\sum_{i \in I}x_{ij}=1, \quad \forall j \in J$

$0 \le x_{ij} \le 1, \quad \forall i,j \in I,J$


Look at your solution. If the solution consists of one cycle, then you are done. For instance, suppose we have a problem with 4 customers and obtain the following solution:

$x_{12} = 1$, $x_{23} = 1$, $x_{34} = 1$, $x_{41} = 1$

This solution can be represented as a single solution, so we have found the optimal solution

If we end up with a solution that looks like:

$x_{12} = 1$, $x_{21} = 1$, $x_{34} = 1$, $x_{43} = 1$

Then we need to go to step 2

### 2. Add subtour elmination constraints and resolve

The solution above shows that we have 2 subtours. In order to find a solution to the TSP, we need to add subtour elimination constraints. For this particular example, we would add the following constraints:

$x_{12} + x_{21} \le 1 \newline$ 
$x_{34} + x_{43} \le 1$

Additonally, we should explicitly set the decision variables to binary


## Example Problem

Let's solve the example problem in the Google OR-Tools Tutorial: https://developers.google.com/optimization/routing/tsp

We will setup and solve this problem using `docplex`. `docplex` is a library that allows you to use CPLEX (a commercial mathematical programming solver from IBM) in Python. This package requires an installation of CPLEX. You can download a free, limited edition of CPLEX from IBM.

### Imports

In [ ]:
from docplex.mp.model import Model

### Input Data

In [ ]:
# list of cities
cities = ['NY',
'LA',
'CHI',
'MIN',
'DEN',
'DAL',
'SEA',
'BOS',
'SA',
'STL',
'HOU',
'PHX',
'SLC']



In [ ]:
# list of arcs connecting cities
arcs = [(i,j) for i in cities for j in cities if i != j]

In [ ]:
# copy distance data from Google
distance_matrix= [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]

# convert to a single list and remove 0s
distances = [ele for sub_list in distance_matrix for ele in sub_list if ele != 0]

# create distance matrix dictionary for model
# using c for model notation
c = {(arcs[i]): distances[i] for i in range(len(arcs))}

### Optimization Model

In [ ]:
# create a model object
tsp = Model(name='TSP')

In [ ]:
# create the x_ij binary decision variables
x = tsp.binary_var_dict(keys = arcs, name="x")

In [ ]:
# add the objective function of minimizing total distance
total_distance = tsp.sum(c[(i,j)] * x[(i,j)] for (i,j) in arcs)
tsp.minimize(total_distance)

In [ ]:
# add the constraints

# leave each city exactly once
tsp.add_constraints( ((tsp.sum(x[(i,j)] for j in cities if j != i) == 1), 
f'from_{i}') for i in cities )

# enter each city exactly once
tsp.add_constraints( ((tsp.sum(x[(i,j)] for i in cities if i != j) == 1), 
f'to_{j}') for j in cities )

In [ ]:
# solve the problem
tsp.print_information()

In [ ]:
tsp.solve()

In [ ]:
tsp.solution.objective_value

In [ ]:
# non-zero variables
sol_vars = [x[x_ij] for x_ij in x if x[x_ij].solution_value == 1]

In [ ]:
# loop through until we have only 1 tour
while True:

    # constraints to add
    # a list of lists
    tours = []

    # current subtour
    subtour = []

    # create a dictionary linking each city to its destination
    travel = {x.name.split('_')[1]: (x,x.name.split('_')[2]) for x in sol_vars}

    # start with NY
    current_city = 'NY'

    # list of variables left to check
    remain_cities = cities.copy()

    for num in range(len(sol_vars)):

        # add from current_city variable to the subtour
        subtour.append(travel[current_city][0])

        # eliminate the variable from the remaining cities list
        remain_cities.remove(current_city)

        # check if we have a closed subtour
        if subtour[0].name.split('_')[1] == subtour[-1].name.split('_')[2]:
            
            # add the subtour to our list of tours
            tours.append(subtour)

            # set the subtour to an empty list
            subtour = []

            # set a new current city if we are not at the end of the loop
            if remain_cities != []:
                current_city = remain_cities[0] 

        # if the subtour isn't closed, the current city is the str at the end of the var name
        else:
            current_city = travel[current_city][1]

    # break if we have only 1 tour
    if len(tours) == 1:
        break

    else:

        # add the constraints to the problem
        tsp.add_constraints(  (tsp.sum(subtour) <= (len(subtour) - 1),'subtour_') for subtour in tours)
        
        # resolve the problem
        tsp.solve()

        # print the objective function value
        print(f"Current Obj: {tsp.solution.objective_value}")

        # get list of variables that equal 1
        sol_vars = [x[x_ij] for x_ij in x if x[x_ij].solution_value == 1] 


# finished
# print the objective function value
print(f"Final Obj: {tsp.solution.objective_value}")

    

In [ ]:


my_list = [1,2,3,4,5,6,7,8,9,10]

a = my_list[0]

my_list.remove(1)

print(f"a: {a}")
print(f"my_list: {my_list}")


In [ ]:
baby_list = []

In [ ]:
baby_list_111 = baby_list[0]

In [ ]:
a = [1]
a.append(9)
a

In [ ]:
a[-1]

In [ ]:
travel['NY'][0]

In [ ]:
%%timeit

a = [1]

a.extend([2,3])
a

In [ ]:
%%timeit

a = [1]

a.append(2)
a.append(3)
a

In [ ]:
x[arc].name.split('_')[1]: x[arc].name.split('_')[2] for arc in sol_vars

In [ ]:
[arc.name for arc in sol_vars]